In [1]:
#|default_exp preprocessing
#|export

import polars as pl
import numpy as np

from datetime import datetime
import calendar

import json
import os

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [50]:
#|export

class DataHandler():
    def __init__(self, csv_path: str = '../local_data/all_lending_club_loan_data_2007-2018.csv'):
        self.cleaned_csv_path = f'{csv_path[:-4]}_cleaned.csv'
        self.features_path = f'{csv_path[:-4]}_cleaned_features.json'

        if not os.path.exists(self.cleaned_csv_path):
            self.strip_non_data_rows_from_lending_club_data(csv_path)
            self.clean_lending_club_data()

        with open(self.features_path, 'r') as f:
            self.features = json.load(f)

        self.transformer = self.column_transformer = ColumnTransformer(
            transformers=[
                ('numeric', MinMaxScaler(), self.features['numeric']),
                ('categorical', OneHotEncoder(drop=None), self.features['categorical'] )
            ],
        )

        self.training_data_start_date = None
        self.training_data_end_date = None
    
    def strip_non_data_rows_from_lending_club_data(self, csv_path: str):
        with open(csv_path, 'r') as f:
            lines = [line for line in f if line[0].isdigit() or line.startswith('id')]

        with open('temp.csv', 'w') as f:
            f.writelines(lines)

    def is_date_value(self, value: str) -> bool:
        '''
        Check if a value matches month-year format (e.g., 'sep-2015')
        '''
        if not isinstance(value, str) or '-' not in value:
            return False
        
        try:
            month_str, year_str = value.lower().split('-')
            return (month_str.title() in calendar.month_abbr and 
                    year_str.isdigit() and 
                    len(year_str) == 4)
        except:
            return False
        
    def parse_date_value(self, value: str) -> datetime:
            '''
            Convert month-year string to datetime
            '''
            if not isinstance(value, str):
                return None
            
            try:
                month_str, year_str = value.lower().split('-')
                month_num = list(calendar.month_abbr).index(month_str.title())
                return datetime(int(year_str), month_num, 1)
            except:
                return None
            
    def drop_undesired_columns(self, df: pl.DataFrame)-> pl.DataFrame:
        explicit_columns_to_drop = [
            'id',                       # Unique identifier
            'funded_amnt',              # Redundant due to loan_amnt
            'funded_amnt_inv',          # Redundant due to loan_amnt
            'sub_grade',                # Redundant due to grade column
            'emp_title',                # Too random
            'title',                    # Redundant in relation to purpose column
            'desc',                     # Mostly null
            'url',                      # No predictive value
            'mths_since_last_delinq',   # Mostly null and redundant due to delinq_2yrs column
            'mths_since_last_record',   # Mostly null
            'pymnt_plan',               # Always 'n'
            'addr_state',               # Reduce dimensionality of this excerise
            'zip_code'                  # Reduce dimensionality of this excerise
        ]

        # Dropping all secondary applicant, hardship, settlement, and joint columns as they are all null
        implicit_columns_to_drop = (
            'sec_app_',
            'hardship_',
            'settlement_',
            'joint_',
        )

        implicit_columns_to_drop = [column for column in df.columns if column.startswith(implicit_columns_to_drop)]
        high_null_cols = [column for column in df.columns  if (df[column].null_count() / len(df)) > 0.2]

        columns_to_drop = explicit_columns_to_drop + implicit_columns_to_drop + high_null_cols
        columns_to_drop = list(set(columns_to_drop))
        
        return df.drop(columns_to_drop)

    def convert_employment_length(self, value: str) -> float:
        if value is None or value == 'n/a':
            return None
        if value == '< 1 year':
            return 0.5
        if value == '10+ years':
            return 10.0
        return float(value.split()[0])

    def clean_lending_club_data(self):
        '''
        Process Lending Club data:
        1. Identify null and date (month-year format) columns
        2. Deelete null columns
        2. Convert identified date columns to datetime
        '''

        df = pl.read_csv('temp.csv')

        null_columns = []
        date_columns = []
        
        #Identify null and date columns
        for col in df.columns:
            # Check if column is all null
            if df[col].is_null().all():
                null_columns.append(col)
                continue
            
            #Identify date columns
            sample = df[col].drop_nulls().sample(1)
            if self.is_date_value(str(sample[0])):
                date_columns.append(col)

        df = df.drop(null_columns)

        #Convert date columns to datetime
        if date_columns:
            df = df.with_columns([
                pl.col(col)
                .str.to_lowercase()
                .map_elements(self.parse_date_value, return_dtype=datetime)
                .alias(col)
                for col in date_columns
            ])

            #Include month and year columns for each date column
            expressions = []
            for col in date_columns:
                expressions.extend([
                    pl.col(col).dt.month().alias(f'{col}_month'),
                    pl.col(col).dt.year().alias(f'{col}_year')
                ])
            df = df.with_columns(expressions)

        df = df.with_columns([
            pl.col('term').str.extract(r'(\d+)').cast(pl.Int64).alias('term_months'),
            pl.col('emp_length').map_elements(self.convert_employment_length, return_dtype=float).alias('employment_years'),
            pl.col('grade').str.to_uppercase().map_elements(lambda x: ord(x) - 64, return_dtype=pl.Int64).alias('grade'),
            pl.col('debt_settlement_flag').map_elements(lambda x: 1 if x == 'Y' else 0, return_dtype=pl.Int64).alias('debt_settlement_flag'),
            pl.col('orig_projected_additional_accrued_interest').cast(pl.Float64).alias('orig_projected_additional_accrued_interest')
            ]).drop(['emp_length', 'term'])
        
        df = self.drop_undesired_columns(df)

        os.remove('temp.csv')

        numeric_columns = [column for column, dtype in df.schema.items() 
                           if dtype in (pl.Float64, pl.Int64)]
        categorical_columns = [col for col, dtype in df.schema.items() 
                               if dtype == pl.Utf8]

        features_dict = {
            'numeric': numeric_columns,
            'categorical': categorical_columns
        }

        with open(self.features_path, 'w') as f:
            json.dump(features_dict, f, indent=2)

        df = df.with_columns([
            pl.col(col).fill_null('missing') for col in categorical_columns
        ])

        df.write_csv(self.cleaned_csv_path)

    def get_data_by_date_range(self, start_date: datetime, end_date: datetime, date_column: str = 'issue_d', return_unlisted_columns: bool = False):
        '''
        Extract rows between two datetime values using a lazy frame
        '''
        lf = pl.scan_csv(
            self.cleaned_csv_path, 
            low_memory=True,
            try_parse_dates=True)
        
        filtered_lf = lf.filter(
            pl.col(date_column).is_between(start_date, end_date)
        )

        if return_unlisted_columns:
            return filtered_lf.collect()
        else:
            df = filtered_lf.collect()
            df = filtered_lf.collect()

            datetime_columns = [column for column in df.columns if df[column].dtype == pl.Datetime]

            return df.drop(datetime_columns)

    def get_training_data(self, start_date: datetime, end_date: datetime)-> tuple[np.ndarray, np.ndarray]:
        '''Get transformed data ready for the autoencoder'''
        self.training_data_start_date = start_date
        self.training_data_end_date = end_date

        raw_data = self.get_data_by_date_range(start_date, end_date, date_column='issue_d', return_unlisted_columns=False)
        transformed_data = self.transformer.fit_transform(raw_data)
        
        missing_mask = np.isnan(transformed_data)
        transformed_data[missing_mask] = -1

        return transformed_data, missing_mask

    def get_test_data(self, start_date: datetime, end_date: datetime)-> tuple[np.ndarray, np.ndarray]:
        if self.training_data_start_date is None or self.training_data_end_date is None:
            raise ValueError('Training data not set. Please call get_training_data first.')
        
        if end_date < self.training_data_start_date or start_date > self.training_data_end_date:
            raw_data = self.get_data_by_date_range(start_date, end_date, date_column='issue_d', return_unlisted_columns=False)
            transformed_data = self.transformer.transform(raw_data)
        
            missing_mask = np.isnan(transformed_data)
            transformed_data[missing_mask] = -1

            return transformed_data, missing_mask
            
        else:
            raise ValueError('There is an overlap between the training and test data.')
        
    def get_transformed_data_feature_names(self):
        return self.transformer.get_feature_names_out()


In [62]:
#|test

data_handler = DataHandler(csv_path='../local_data/all_lending_club_loan_data_2007-2018.csv')

start = datetime(2010,1,1)
end = datetime(2015,12,1)
training_data, missing_mask = #|export.get_training_data(start, end)

if len(training_data[0]) != 108:
    raise ValueError()
